In [131]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
import scipy as sp
%matplotlib ipympl

plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=200, formatter={'float': '{:.6f}'.format})
# np.set_printoptions(linewidth=200, formatter={'float': '{:.3e}'.format})

In [132]:
GHz = 1e9
fF = 1e-15
nH = 1e-9

In [133]:
def coupled_resonators(L1, C1, L2, C2, CC):
    L_01 = sq.Inductor (L1, 'nH')
    C_01 = sq.Capacitor(C1, 'fF')
    L_02 = sq.Inductor (L2, 'nH')
    C_02 = sq.Capacitor(C2, 'fF')
    C_12 = sq.Capacitor(CC, 'fF')
    elements = {(0, 1): [L_01, C_01],
                (0, 2): [L_02, C_02],
                (1, 2): [C_12], }
    return sq.Circuit(elements)

def resonator(L, C):
    L_01   = sq.Inductor (L, 'nH')
    C_01   = sq.Capacitor(C, 'fF')
    elements = {(0, 1): [L_01, C_01],}
    return sq.Circuit(elements)


In [134]:
n_eig = 5
trunc_num = 25

In [262]:
L1 = 3
C1 = 3
L2 = 1
C2 = 5

CC = 2

In [263]:
coupled_res_full = coupled_resonators(L1=L1, C1=C1, L2=L2, C2=C2, CC=CC)
coupled_res_full .set_trunc_nums([trunc_num, trunc_num])
H_coupled_res_full = coupled_res_full.hamiltonian()

In [264]:
C_mat = np.array([[C1 + CC, -CC],
                  [-CC, C2 + CC]])
C_inv = np.linalg.inv(C_mat)
C1_tilde = C_inv[0,0]**-1
C2_tilde = C_inv[1,1]**-1

In [265]:
resonator_1 = resonator(L=L1, C=C1_tilde)
resonator_2 = resonator(L=L2, C=C2_tilde)
resonator_1.set_trunc_nums([trunc_num])
resonator_2.set_trunc_nums([trunc_num])

In [266]:
Q_1 = resonator_1.charge_op(0)
Q_2 = resonator_2.charge_op(0)
Φ_1 = resonator_1.flux_op(0)
Φ_2 = resonator_2.flux_op(0)

# H_1 = resonator_1.hamiltonian()
H_1 = 1/2 * C1_tilde**-1 * fF**-1 * Q_1**2 + 1/2 * L1**-1 * nH**-1 * Φ_1**2 
# H_2 = resonator_2.hamiltonian()
H_2 = 1/2 * C2_tilde**-1 * fF**-1 * Q_2**2 + 1/2 * L2**-1 * nH**-1 * Φ_2**2 

I = qt.identity(H_1.shape[0])
H_coupled_res_comp = qt.tensor(H_1,I) + qt.tensor(I,H_2) + C_inv[0,1]* fF**-1 * qt.tensor(Q_1,Q_2)

In [267]:
E_coupled_res_comp = sq_ext.diag(H_coupled_res_comp, n_eig, remove_ground=True)[0]
E_coupled_res_full = sq_ext.diag(H_coupled_res_full, n_eig, remove_ground=True)[0]

In [268]:
E_coupled_res_comp, E_coupled_res_full

(array([0.000000, 39.431161, 66.613036, 78.862321, 106.044197]),
 array([0.000000, 39.431161, 66.613036, 78.862321, 106.044197]))